# Neighborhood Computations

In [ ]:
import numpy as np
import igl
import meshplot

In [ ]:
bunny_v, bunny_f = igl.read_triangle_mesh("data/bunny.off")
cube_v, cube_f = igl.read_triangle_mesh("data/cube.obj")
sphere_v, sphere_f = igl.read_triangle_mesh("data/sphere.obj")

In [ ]:
meshplot.plot(bunny_v, bunny_f, shading={"wireframe": True})

In [ ]:
meshplot.plot(cube_v, cube_f, shading={"wireframe": True})

In [ ]:
meshplot.plot(sphere_v, sphere_f, shading={"wireframe": True})

## Vertex-to-Face Relations

## Vertex-to-Vertex Relations

## Shading

Meshplot requires per vertex normals, so we need to "explode" the mesh

### Flat Shading

### Per-vertex Shading

### Per-corner Shading

## Connected Components

## A simple subdivision scheme